# Most Dangerous Streets - Crash Leaderboard

Compare the most recent month to the all-time numbers to see which streets got safer. 

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2020,7,1))
end_date = cda.most_recent_crash_timestamp()

all_time = cda.street_metrics(start_date, end_date)

In [4]:
start_date = pytz.timezone('America/Denver').localize(datetime(2023,3,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,4,1))

last_month = cda.street_metrics(start_date, end_date)

In [5]:
# A street must have at least 100 crashes all-time to be in the comparison
all_time = all_time[all_time.num_crashes >= 100].copy()

In [15]:
all_time

,gid,lrsroute,fullname,length_miles,street_line,num_crashes,num_sbi,num_fatality,num_sbi_or_fatality,days_in_data,days_between_crashes,crashes_per_mile_per_week
1535,3900,SPEERN BD,N SPEER BLVD,2.634723,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",642,20,5,25,1007,1.568536,1.693825
1710,4229,22ND ST,22ND ST,1.079568,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",206,4,2,6,1007,4.888350,1.326435
2344,5587,PARK AV,PARK AVE W,1.809664,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",345,12,3,15,1007,2.918841,1.325225
524,1012,COLFAX AV,E COLFAX AVE,5.475202,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",990,47,5,52,1007,1.017172,1.256908
452,880,BROADWAY3,N BROADWAY,3.331934,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",592,20,9,29,1007,1.701014,1.235076
...,...,...,...,...,...,...,...,...,...,...,...,...
448,875,BRIGHTON BD,N BRIGHTON BLVD,2.807560,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",104,4,2,6,1007,9.682692,0.257497
3098,7738,SLOGAN ST,S LOGAN ST,3.377398,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",123,2,0,2,1007,8.186992,0.253158
324,572,56TH AV,E 56TH AVE,9.027374,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",296,12,3,15,1007,3.402027,0.227929
109,174,23RD AV1,E 23RD AVE,4.660338,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",145,4,1,5,1007,6.944828,0.216281


In [6]:
comparison = pd.merge(
    all_time
    , last_month
    , how='inner'
    , on=['gid', 'lrsroute', 'fullname', 'length_miles','street_line']
    , suffixes=['_all_time', '_last_month']
)

In [7]:
comparison['diff_pmpw'] = (
    comparison['crashes_per_mile_per_week_last_month']
    - comparison['crashes_per_mile_per_week_all_time']
)

In [8]:
comparison['expected_crashes'] = (
    comparison['length_miles']
    * comparison['crashes_per_mile_per_week_all_time']
    * (comparison['days_in_data_last_month'] / 7)
)

comparison['diff_expected_crashes'] = comparison['num_crashes_last_month'] - comparison['expected_crashes']

## Difference in Crash Rate per Mile

In [9]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_pmpw').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_pmpw').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [10]:
comparison.sort_values(by='diff_pmpw')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'num_crashes_all_time'
    , 'num_crashes_last_month'
    , 'expected_crashes'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,num_crashes_all_time,num_crashes_last_month,expected_crashes,diff_expected_crashes
20,3888,N SANTA FE DR,1.732911,0.906569,0.269296,-0.637272,226,2,6.732870,-4.732870
7,4225,20TH ST,1.453330,1.200544,0.642203,-0.558341,251,4,7.477656,-3.477656
30,7725,S LINCOLN ST,1.069023,0.760795,0.218268,-0.542527,117,1,3.485601,-2.485601
48,3813,WELTON ST,1.769275,0.573622,0.131881,-0.441742,146,1,4.349553,-3.349553
68,5589,N PECOS ST,2.001631,0.468833,0.116572,-0.352262,135,1,4.021847,-3.021847
...,...,...,...,...,...,...,...,...,...,...
42,866,BLAKE ST,2.422119,0.614167,1.156012,0.541845,214,12,6.375372,5.624628
27,136,E 18TH AVE,1.582608,0.834543,1.474359,0.639816,190,10,5.660377,4.339623
9,5588,PARK AVE W,0.651755,1.141216,1.790038,0.648822,107,5,3.187686,1.812314
1,4229,22ND ST,1.079568,1.326435,2.161359,0.834924,206,10,6.137041,3.862959


## Difference in Number of Expected Crashes

In [11]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_expected_crashes').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_expected_crashes').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [12]:
comparison.sort_values(by='diff_expected_crashes')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'expected_crashes'
    , 'num_crashes_last_month'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,expected_crashes,num_crashes_last_month,diff_expected_crashes
60,6481,S SHERIDAN BLVD,5.380667,0.536143,0.216826,-0.319317,12.363456,5,-7.363456
12,1016,N COLORADO BLVD,5.184503,1.032410,0.765101,-0.267309,22.939424,17,-5.939424
51,2474,N QUEBEC ST,5.785164,0.565944,0.362997,-0.202947,14.031778,9,-5.031778
20,3888,N SANTA FE DR,1.732911,0.906569,0.269296,-0.637272,6.732870,2,-4.732870
7,4225,20TH ST,1.453330,1.200544,0.642203,-0.558341,7.477656,4,-3.477656
...,...,...,...,...,...,...,...,...,...
0,3900,N SPEER BLVD,2.634723,1.693825,2.125461,0.431635,19.126117,24,4.873883
42,866,BLAKE ST,2.422119,0.614167,1.156012,0.541845,6.375372,12,5.624628
64,2939,N TOWER RD,2.510320,0.481824,1.022446,0.540622,5.183714,11,5.816286
2,5587,PARK AVE W,1.809664,1.325225,2.320873,0.995648,10.278054,18,7.721946


In [13]:
comparison = comparison.sort_values(by='diff_expected_crashes')

comparison.loc[comparison['diff_expected_crashes'].rank() <= 5, 'leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank() <= 5].apply(
        lambda row: f'{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5, 'leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5].apply(
        lambda row: f'+{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)


In [14]:
comparison['street_name'] = comparison['fullname'].str.title()
comparison.loc[comparison.leaderboard.notnull(), ['street_name', 'leaderboard']]

,street_name,leaderboard
60,S Sheridan Blvd,-7 crashes
12,N Colorado Blvd,-6 crashes
51,N Quebec St,-5 crashes
20,N Santa Fe Dr,-5 crashes
7,20Th St,-3 crashes
0,N Speer Blvd,+5 crashes
42,Blake St,+6 crashes
64,N Tower Rd,+6 crashes
2,Park Ave W,+8 crashes
19,S Federal Blvd,+9 crashes
